Os seguintes valores são calculado com o histograma e são largamente utilizados para representar imagens:
- Média
- Variância
- Skewness
- Kurtosis
- Energia
- Entropia



In [ ]:
#dependencias necessárias
!python -m pip install -U scikit-image
!python -m pip install -U scikit-learn
!python -m pip install -U scipy
!python -m pip install -U pandas
!python -m pip install -U opencv-python
!python -m pip install -U openpyxl
!python -m pip install -U matplotlib




In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.metrics import accuracy_score
import numpy as np
from glob import glob
import pandas as pd
from skimage.io import imread
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:

def descritor_histograma(img_rgb):
    """
    Recebe uma imagem RGB como parâmetro
    """
    import numpy as np

    # Defina o número de intervalos de intensidade (256 para imagens em 8 bits)
    num_bins = 256

    # Inicialize o histograma para cada canal de cor
    hist_r = np.zeros(num_bins, dtype=int)
    hist_g = np.zeros(num_bins, dtype=int)
    hist_b = np.zeros(num_bins, dtype=int)

    # Calcule o histograma manualmente para cada canal
    for row in img_rgb:
        for pixel in row:
            hist_r[pixel[0]] += 1
            hist_g[pixel[1]] += 1
            hist_b[pixel[2]] += 1

    # Calcule a média para cada canal
    mean_r = np.sum([i * hist_r[i] for i in range(num_bins)]) / np.sum(hist_r)
    mean_g = np.sum([i * hist_g[i] for i in range(num_bins)]) / np.sum(hist_g)
    mean_b = np.sum([i * hist_b[i] for i in range(num_bins)]) / np.sum(hist_b)

    media = mean_r + mean_g + mean_b

    # Calcule a variância para cada canal
    variance_r = np.sum([(i - mean_r) ** 2 * hist_r[i] for i in range(num_bins)]) / np.sum(hist_r)
    variance_g = np.sum([(i - mean_g) ** 2 * hist_g[i] for i in range(num_bins)]) / np.sum(hist_g)
    variance_b = np.sum([(i - mean_b) ** 2 * hist_b[i] for i in range(num_bins)]) / np.sum(hist_b)

    var = variance_r + variance_g + variance_b

    # Calcule a skewness para cada canal
    skewness_r = np.sum([((i - mean_r) / np.sqrt(variance_r)) ** 3 * hist_r[i] for i in range(num_bins)]) / np.sum(hist_r)
    skewness_g = np.sum([((i - mean_g) / np.sqrt(variance_g)) ** 3 * hist_g[i] for i in range(num_bins)]) / np.sum(hist_g)
    skewness_b = np.sum([((i - mean_b) / np.sqrt(variance_b)) ** 3 * hist_b[i] for i in range(num_bins)]) / np.sum(hist_b)

    sk = skewness_r + skewness_g + skewness_b

    # Calcule a kurtosis para cada canal
    kurtosis_r = np.sum([((i - mean_r) / np.sqrt(variance_r)) ** 4 * hist_r[i] for i in range(num_bins)]) / np.sum(hist_r)
    kurtosis_g = np.sum([((i - mean_g) / np.sqrt(variance_g)) ** 4 * hist_g[i] for i in range(num_bins)]) / np.sum(hist_g)
    kurtosis_b = np.sum([((i - mean_b) / np.sqrt(variance_b)) ** 4 * hist_b[i] for i in range(num_bins)]) / np.sum(hist_b)

    kur = kurtosis_r + kurtosis_g + kurtosis_b

    # Calcule a energia para cada canal
    energy_r = np.sum(hist_r ** 2)
    energy_g = np.sum(hist_g ** 2)
    energy_b = np.sum(hist_b ** 2)

    ener = energy_r + energy_g + energy_b

    # Calcule a entropia para cada canal
    hist_normalized_r = hist_r / np.sum(hist_r)
    entropy_r = -np.sum(hist_normalized_r * np.log2(hist_normalized_r + np.finfo(float).eps))

    hist_normalized_g = hist_g / np.sum(hist_g)
    entropy_g = -np.sum(hist_normalized_g * np.log2(hist_normalized_g + np.finfo(float).eps))

    hist_normalized_b = hist_b / np.sum(hist_b)
    entropy_b = -np.sum(hist_normalized_b * np.log2(hist_normalized_b + np.finfo(float).eps))

    entr = entropy_r + entropy_g + entropy_b


    return [media,var,sk,kur,ener,entr]



- Procurar uma base de imagens com duas classes ou mais
- Calcular os atributos de primeira ordem para cada imagem, e criar uma matriz, e adicionar uma coluna na matriz para os rótulos das classes. 
- Plotar os atributos dois a dois (scatter plot)
- A entrega será:
    - Notebook com os códigos;
    - Short paper de até 4 páginas com:
        - Introdução: apresentar o problema/base escolhida;
        - Metodologia: explicar os atributos e como são calculados;
        - Resultados: resultado da classificação e scatter plots dos atributos;
        - Extra: plotar os atributos utilizando um algoritmo de redução de dimensionalidade, ex: PCA, t-SNE;
        - Conclusão.
- Será feito pela equipe definida no inicio da disciplina;
- Entrega e apresentação: 25/10/2023

In [ ]:


def classification(X,y):

    """
    X é uma matriz onde cada linha corresponde aos atributos calculados com o descritor
    y são os rótulos da classe de cada imagem. Exemplo [0,1,0,0,1,2,2,0,0,0,...,1]
    """
    
    # separar conjuntos de treino e teste
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7)
    # criar instancia do classificador
    clf = mlp()
    # treinar o classificador
    clf.fit(X_train,y_train)
    # realizar a predicao
    pred = clf.predict(X_test)
    # calcular o resultado utilizando a acuracia
    acuracia = accuracy_score(y_test,pred)

    
    
    return np.round(acuracia,decimals=2) 

# Processamento dos dados

In [ ]:

data = []

dirs = "dados/*.jpg"
excel_name = 'dados.xlsx'
column_names = ['fname', 'deworm', 'mean', 'var', 'skew', 'kurtosis', 'energy', 'entropy']

fnames = glob(dirs)

for name in fnames:
    
    vermifuga = 0
    
    famacha = int(name[-5])
    if famacha > 2:
        vermifuga = 1
    
    img = imread(name)
    
    aux = descritor_histograma(img)
    aux.insert(0,name[6:])
    aux.insert(1,vermifuga)
    data.append(aux)
    
print(data)

df = pd.DataFrame(data,columns=column_names)

df.to_excel(excel_name, index=False)

# Carregamento dos dados já processados

In [ ]:
# Carregar o arquivo Excel em um DataFrame
df = pd.read_excel('dados.xlsx')


matriz = []

#salvando os dados em uma matriz
for i in range(0,len(df),1):
    matriz.append([df["mean"][i],df["var"][i],df["skew"][i],df["kurtosis"][i], df["energy"][i],df["entropy"][i]])

famacha = df["deworm"]


#print(classification(matriz, famacha))


# Criação dos Scatter plots 2 a 2

In [ ]:
output = "scatter_plots"

if not os.path.exists(output):
    os.makedirs(output)

columns = ["mean", "var", "skew", "kurtosis", "energy", "entropy"]

for i in range(0, len(columns) - 1, 1):
    for j in range(i + 1, len(columns), 1):
        plt.figure(figsize=(8, 6))

        plt.scatter(df[columns[i]], df[columns[j]], c='blue', label=columns[i])
        plt.scatter(df[columns[i]], df[columns[j]], c='black', label=columns[j])
        
        plt.xlabel(columns[i])
        plt.ylabel(columns[j])
        plt.legend()

        fig_name = f'{columns[i]}_x_{columns[j]}.png'
        plt.savefig(os.path.join(output, fig_name))
        plt.close()  # Fechar a figura para evitar sobreposição ao criar a próxima


# Procurando padrões

In [ ]:
lista = []
for i in range(0,1000,1):
    lista.append(classification(matriz, famacha))
    

print(f"\nMaior Valor: {max(lista)}\nMédia: {np.mean(lista)}\nMenor: {min(lista)}")

# Aplicando Transformação de Intensidade


In [3]:
def alargamento_contraste(nome_imagem:str, k:float = 1, E:float = 1)->np.ndarray:
    """
    Realiza o Alargamento de contraste de uma imagem
    
    Parâmetros:
        nome_imagem::str: Nome da imagem que será lida e processada
        k::float: controla a intensidade da transformação. Quanto maior o valor de k, mais intensa será a transformação, onde k afeta a amplitude da função de transformação.Se k for pequeno, a transformação terá um efeito mais suave.
        E::float: Expoente controla a forma da transformação. Se E for um valor baixo, a transformação será mais linear. Se E for um valor mais alto, a transformação será menos linear. Valores de E maiores resultam em uma transformação mais agressiva nos extremos dos valores de pixel, enquanto valores menores de E proporcionam uma transformação mais suave.
        
    Retorno:
        imagem_transformada::numpy.ndarray: Imagem transformada pelo apargamento de contraste
    """
    
    imagem = imread(nome_imagem)
    # Adiciona uma constante pequena para evitar log(0) e/ou divisão por zero
    epsilon = 1e-8
    
    # Aplica a transformação de alargamento de contraste
    imagem_transformada = 1 / (1 + (k / (imagem+epsilon))**E)
    
    # Garante que os valores estejam no intervalo [0, 1] (apenas int, nada de float) 
    imagem_transformada = np.clip(imagem_transformada, 0, 255)#.astype(np.uint8)
    
    return imagem_transformada

def negativo(nome_imagem:str)->np.ndarray:
    """
    Transforma uma imagem para sua versão negativa
    
    Parâmetros:
        img::string: Nome de uma imagem
        
    Retorno:
        neg::np.array: Imagem negativada 
        
    """
    imagem = imread(nome_imagem)
    """
    realiza a inversão do valor e o converte para seu valor inveso absoluto (ignora o sina);
    """
    neg = abs(255-imagem)
    
    return neg

def logaritmo(nome_imagem:str, c:float=1)->np.ndarray:
    """
    Calcula a transformação de intensidade logarítma da imagem
    
    Parâmetros:
        nome_imagem::str: Nome da imagem que será transformada
        c::float: coeficiente que multiplica o resultado da transformação logarítmica
    Retorno:
        logaritmico_img::np.ndarray: Imagem gerada apartir do calculo do logartimo 
    """
    img = imread(nome_imagem)
    
    # Adiciona uma constante pequena para evitar log(0)
    epsilon = 1e-8
    
    # Aplica a transformação logarítmica para cada canal RGB
    logaritmico_img = c * np.log(1 + img + epsilon)
    
    # Garante que os valores estejam no intervalo [0, 255] e que sejam inteiros
    logaritmico_img = np.clip(logaritmico_img, 0, 255).astype(np.uint8)
    
    return logaritmico_img

def transformacao_potencia(nome_imagem:str, c:float=1.0, gamma:float=1.0)->np.ndarray:
    """
    Calcula a transformação de pontencia de uma imagem e gera uma nova imagem com base nisso
    
    Parâmetros:
        nome_imagem::str: Nome da imagem que será transformada
        c::float: coeficiente que multiplica o resultado da transformação de potência. Ele ajusta a amplitude ou intensidade da transformação.
        gamma::float: controla a forma da curva da transformação de potência.
    """
    imagem = imread(nome_imagem)
    
    # Aplica a transformação de potência
    imagem_transformada = c * np.power(imagem, gamma)
    
    # Garante que os valores estejam no intervalo [0, 255]
    imagem_transformada = np.clip(imagem_transformada, 0, 255).astype(np.uint8)
    
    return imagem_transformada